##### Copyright 2021 The TensorFlow Authors.

In [ ]:
#@title Licensed under the Apache License, Version 2.0 (the "License");
# you may not use this file except in compliance with the License.
# You may obtain a copy of the License at
#
# https://www.apache.org/licenses/LICENSE-2.0
#
# Unless required by applicable law or agreed to in writing, software
# distributed under the License is distributed on an "AS IS" BASIS,
# WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
# See the License for the specific language governing permissions and
# limitations under the License.

# Migrating your TPUEstimator to TF2

<table class="tfo-notebook-buttons" align="left">
  <td>
    <a target="_blank" href="https://www.tensorflow.org/guide/migrate/tpu_estimator">
    <img src="https://www.tensorflow.org/images/tf_logo_32px.png" />
    View on TensorFlow.org</a>
  </td>
  <td>
    <a target="_blank" href="https://colab.research.google.com/github/tensorflow/docs/blob/master/site/en/guide/migrate/tpu_estimator.ipynb">
    <img src="https://www.tensorflow.org/images/colab_logo_32px.png" />
    Run in Google Colab</a>
  </td>
  <td>
    <a target="_blank" href="https://github.com/tensorflow/docs/blob/master/site/en/guide/migrate/tpu_estimator.ipynb">
    <img src="https://www.tensorflow.org/images/GitHub-Mark-32px.png" />
    View source on GitHub</a>
  </td>
  <td>
    <a href="https://storage.googleapis.com/tensorflow_docs/docs/site/en/guide/migrate/tpu_estimator.ipynb"><img src="https://www.tensorflow.org/images/download_logo_32px.png" />Download notebook</a>
  </td>
</table>

In TF1, `tf.compat.v1.estimator.tpu.TPUEstimator` is a high level API that encapsulates training, evaluation, prediction, and export for serving with TPUs. In TF2, use `tf.keras` with a Keras [model](https://www.tensorflow.org/api_docs/python/tf/keras/Model?version=nightly), [layers](https://www.tensorflow.org/api_docs/python/tf/keras/layers/Layer?version=nightly), [optimizers](https://www.tensorflow.org/api_docs/python/tf/keras/optimizers/Optimizer?version=nightly), and [metrics](https://www.tensorflow.org/api_docs/python/tf/keras/metrics/Metric) for the aforementioned tasks. In this guide, you will go through the basic setup of a `TPUEstimator` training and evaluation program to run it. Then, you will create the equivalent training and evaluation program in TF2 using Keras, with the help of `tf.distribute.TPUStrategy`.

## Setup

For both TF1 and TF2 examples demonstrated below, you will first start with a couple of necessary TensorFlow imports,

In [ ]:
import tensorflow as tf
import tensorflow.compat.v1 as tf1

and prepare some simple data for demonstration:

In [ ]:
features = [[1., 1.5]]
labels = [[0.3]]
eval_features = [[4., 4.5]]
eval_labels = [[0.8]]

## TF1: TPUEstimator.train/evaluate

Now, to use a `TPUEstimator`, you should first define a few functions: an input function for the training data, an evaluation input function for the evaluation data, and a model function that tells the `TPUEstimator` how the training op is defined with the features and labels.

In [ ]:
def _input_fn(params):
  dataset = tf1.data.Dataset.from_tensor_slices((features, labels))
  dataset = dataset.repeat()
  return dataset.batch(params['batch_size'], drop_remainder=True)

def _eval_input_fn(params):
  dataset = tf1.data.Dataset.from_tensor_slices((eval_features, eval_labels))
  dataset = dataset.repeat()
  return dataset.batch(params['batch_size'], drop_remainder=True)

def _model_fn(features, labels, mode, params):
  logits = tf1.layers.Dense(1)(features)
  loss = tf1.losses.mean_squared_error(labels=labels, predictions=logits)
  optimizer = tf1.train.AdagradOptimizer(0.05)
  train_op = optimizer.minimize(loss, global_step=tf1.train.get_global_step())
  return tf1.estimator.tpu.TPUEstimatorSpec(mode, loss=loss, train_op=train_op)

With those functions defined, you will now create a `tf.distribute.cluster_resolver.TPUClusterResolver` that provides the cluster information, and a `tf.compat.v1.estimator.tpu.RunConfig` object. Along with the model function you have defined, you can now create a `TPUEstimator`. Note that here you will simplify the flow by skipping checkpoint savings, and need to specify the batch size for both training and evaluation for a `TPUEstimator`.

In [ ]:
cluster_resolver = tf1.distribute.cluster_resolver.TPUClusterResolver(tpu='')
print("All devices: ", tf1.config.list_logical_devices('TPU'))

In [ ]:
tpu_config = tf1.estimator.tpu.TPUConfig(iterations_per_loop=10)
config = tf1.estimator.tpu.RunConfig(
    cluster=cluster_resolver,
    save_checkpoints_steps=None,
    tpu_config=tpu_config)
estimator = tf1.estimator.tpu.TPUEstimator(
    model_fn=_model_fn, config=config, train_batch_size=8, eval_batch_size=8)

Call `train` to train the `TPUEstimator`,

In [ ]:
estimator.train(_input_fn, steps=1)

And call `evaluate` to evaluate the model using the evaluation data you have prepared.

In [ ]:
estimator.evaluate(_eval_input_fn, steps=1)

### TF2: Keras training API with `tf.distribute.TPUStrategy`

In TF2, you will use a `tf.keras` model with a `tf.distribute.TPUStrategy` to train your models on the TPU workers. To begin with, you will create a `TPUClusterResolver` to provide the cluster information, and connect to the cluster.

In [ ]:
cluster_resolver = tf.distribute.cluster_resolver.TPUClusterResolver(tpu='')
tf.config.experimental_connect_to_cluster(cluster_resolver)
tf.tpu.experimental.initialize_tpu_system(cluster_resolver)
print("All devices: ", tf.config.list_logical_devices('TPU'))

Next, once your data is prepared, you will create a `TPUStrategy`, and define your model, metrics, and optimizer under the scope of this strategy. You should pick a number for `steps_per_execution` in `Model.compile` because it specifies the number of batches to run during each `tf.function` call, and is critical for performance. This argument is similar to `iterations_per_loop` used in `TPUEstimator`.

In [ ]:
dataset = tf.data.Dataset.from_tensor_slices(
    (features, labels)).shuffle(10).repeat().batch(8).prefetch(2)
eval_dataset = tf.data.Dataset.from_tensor_slices(
      (eval_features, eval_labels)).batch(1)

strategy = tf.distribute.TPUStrategy(cluster_resolver)
with strategy.scope():
  model = tf.keras.models.Sequential([tf.keras.layers.Dense(1)])
  optimizer = tf.keras.optimizers.Adagrad(learning_rate=0.05)
  model.compile(optimizer, "mse", steps_per_execution=10)

With that, you are ready to train the model with the training dataset,

In [ ]:
model.fit(dataset, epochs=5, steps_per_epoch=10)

and evaluate the model using the evaluation dataset.

In [ ]:
model.evaluate(eval_dataset, return_dict=True)

For more customization such as optimization procedure, Keras APIs allow you to define the training step. Please see [Customize what happens in Model.fit](https://www.tensorflow.org/guide/keras/customizing_what_happens_in_fit
) for an example of this workflow, and [Use TPUs](https://www.tensorflow.org/guide/tpu) for more information of TPU usage, including an example of customized training loop.